In [ ]:
import requests #istek kutuphanesi
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium import webdriver #selenium kutuphanesi
from selenium.webdriver.common.keys import Keys # selenium kutuphanesinden kullanılabilecek anahtarlar
from bs4 import BeautifulSoup #html okuyan kutuphane
import pandas as pd  # dosya islemleri icin pandas kutuphanesi
import datetime #tarih kutphanesi
import time #zaman aralıkları atmak için zaman kuphanesi 
from openpyxl import load_workbook
import numpy as np

r = requests.get('https://seffaflik.epias.com.tr/transparency/piyasalar/gop/arz-talep.xhtml') #websitesini oku 
source = BeautifulSoup(r.content,"lxml") #okuma bicimi olarak xml kullan

source.find("input",attrs={"id":"j_idt219:txt1"}) #input etiketli id si bu olan eleman

tarih = source.find("input",attrs={"id":"j_idt219:date1_input"})["value"] # input etiketli id si bu olanı getir bu da tarih
x = datetime.datetime.now() #şu anki tarihi al 
today = datetime.date.today() # bugunu al 
tomorrow = today + datetime.timedelta(days = 1)  # yarının tarihi
tomorrow = str(tomorrow)# string ifadeye cevir
words = tarih.split('.')  # noktaların oldugu yerlerden ifadeleri bol orn: 03.02.2021 ayrılmıs hali [03,02,2021]
yeni_tarih = '.'.join(reversed(words)) # bu ayrılan tarihi terse çevir 2021,02,03
yeni_tarih =yeni_tarih.replace(".","-") #nokta olan yeri - yap 2021-02-03

def tablo_cek(): #fonksiyon tanımlama
    source = BeautifulSoup(driver.page_source,"lxml")  #<-- get the current html
    columns = ['Price', 'Demand',  'Supply']
    rows = []
    trs = source.find_all('tr')
    for row in trs:
        data = row.find_all('td')
        data = [x.text for x in data]
        rows.append(data)
    rows= rows.append(rows)
    df =pd.DataFrame(rows,columns=columns)
  
    display(df)
    path = "ArzTalep.xlsx"
    book = load_workbook(path)
    writer = pd.ExcelWriter(path, engine = 'openpyxl')
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df1 = pd.DataFrame(df)
    df1.to_excel(writer)
    writer.save()
    
    
   
    
    return source

def saati_tikla():
                    num =0
                    tablo_cek()
                    while num < 1: 
                        button_next = driver.find_element_by_xpath( "//*[@id='j_idt219:dt_paginator_bottom']/a[3]")
                        if not 'ui-state-disabled' in button_next.get_attribute('class'):
                            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='j_idt219:dt_paginator_bottom']/a[3]"))).click()
                            time.sleep(1)
                            #driver.execute_script("arguments[0].click();", button_next)
                            tablo_cek() # gelen websitesindeki tabloyu al ,kaydet 
                        if 'ui-state-disabled' in button_next.get_attribute('class'):
                           
                            time.sleep(1)  
                            user = driver.find_element_by_name("j_idt219:txt1") #tekrar getiriyor saat yerini 
                            user.clear() #saati sıfırla
                            num = num + 1 #saati bir arttır
                            time.sleep(1)
                            print("SAAT:")
                            print(num)
                            user.send_keys(num) #saate veri gönder
                            time.sleep(1)
                            driver.find_element_by_id('j_idt219:goster').click() #saati uygula
                            
                            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id='j_idt219:dt_paginator_bottom']/a[1]/span"))).click()
                    while num >= 1:
                        driver.quit()
                        break;
                        
#sayfa biene kadar donsun 
###********* DONGU BASLIYOR ******######

if tomorrow == yeni_tarih :# sayfadaki tarih guncel mi 
    print(yeni_tarih == tomorrow) #guncelse dogrula
    
    driver = webdriver.Chrome("C:/Users/tugba.ozkan/AppData/Local/SeleniumBasic/chromedriver.exe")#web sürücüsünün nerede oldugu
    driver.get("https://seffaflik.epias.com.tr/transparency/piyasalar/gop/arz-talep.xhtml") #web sürücüsü bu siteyi okusun 
    time.sleep(1)#1 sn ara
    driver.find_element_by_xpath("//select/option[@value='96']").click()#alttaki seçenekten gorunen veri sayısını 96 yap
    #user = driver.find_element_by_name("j_idt219:txt1") #saati girdiğimiz alanı tanımla
    
    time.sleep(1)#1 sn ara
     #tablo_cek() #<-- need to get that data before moving to next page
   
        #WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a/span[@class ='ui-icon ui-icon-seek-next']")))
    saati_tikla()
        
                
else:
    print("Güncelleme gelmedi") # eger epiaştaki tarih yarın degilse guncel degildir bunu göster.

